<a href="https://colab.research.google.com/github/TanyaAgrwal/Machine-learning/blob/main/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Fake news dataset/train.csv')
df.head()

In [ ]:
df.shape

In [ ]:
class_divisions=df.value_counts(df['label'])
class_divisions

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

In [ ]:
title=df['title']

In [ ]:
title

In [ ]:
import re
from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer
#ps=PorterStemmer()
title_data=[]
for i in range(len(title)):
  text=re.sub(r'[^A-Za-z0-9%$]',' ',str(title[i]))
  text=text.strip()
  text=text.split()
  text=[i for i in text if not i in stopwords.words('english')]
  text=' '.join(text)
  title_data.append(text)

In [ ]:
#practice
text1='my name is Tanya Agrawal, I got 90% in maths and @ ! is good in P.T. and i got $100000'
text1=re.sub(r'[^A-Za-z0-9%$]',' ',text1)
text1=text1.split()
text1=' '.join(text1)
text1

In [ ]:
title_data

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train,X_test,y_train,y_test=train_test_split(title_data, df['label'].values, test_size=0.3)

In [ ]:
df2=pd.read_csv('/content/drive/MyDrive/Fake news dataset/test.csv')
df2.head()

In [ ]:
title2=df2['title']

In [ ]:
title2

In [ ]:
import re
from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer
#ps=PorterStemmer()
title_data2=[]
for i in range(len(title2)):
  text2=re.sub(r'[^A-Za-z0-9%$]',' ',str(title2[i]))
  text2=text2.strip()
  text2=text2.split()
  text2=[i for i in text2 if not i in stopwords.words('english')]
  text2=' '.join(text2)
  title_data2.append(text2)

In [ ]:
title_data2

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
sub=pd.read_csv('/content/submit.csv')
sub.head()

In [ ]:
x=title_data+title_data2
a=df['label'].values
b=sub['label'].values
y=np.concatenate((a,b))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y, test_size=0.2, stratify=y)

In [ ]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_vocab=21628
tokenizer= Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(X_train)

In [ ]:
wordindex=tokenizer.word_index
len(wordindex)

In [ ]:
train_seq=tokenizer.texts_to_sequences(X_train)
test_seq=tokenizer.texts_to_sequences(X_test)

In [ ]:
pad_train=pad_sequences(train_seq)
pad_train.shape

In [ ]:
pad_test=pad_sequences(test_seq, maxlen=54)
pad_test.shape

In [ ]:
from keras import Sequential
from keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Flatten, Dropout, BatchNormalization
model=Sequential()
model.add(Embedding(22862,128,input_length=54))
model.add(Dropout(0.3))
model.add(LSTM(80, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(pad_train, y_train,batch_size=100, epochs=2 ,validation_data=(pad_test,y_test))

In [ ]:
history.history

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('accuracy')
plt.legend(['train','test'])
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['accuracy'])
plt.title('loss')
plt.legend(['train','test'])
plt.show()

In [ ]:
model.evaluate(pad_test,y_test)

In [ ]:
y_pred=model.predict_classes(pad_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm=confusion_matrix(y_test,y_pred)
a_s=accuracy_score(y_test,y_pred)
print(cm)
print(a_s)